# Anime Recommender with Cosine Similarity 

In [1]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings # warning filter
import scipy as sp #pivot egineering

In [2]:
import sys
!{sys.executable} -m pip install scipy

In [3]:
#ML model
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#default theme and settings
pd.options.display.max_columns

20

In [5]:
#warning hadle
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [12]:
#download the data set of rating 
rating_df = pd.read_csv("C:\\Users\\Syoto\\Desktop\\ANIME\\rating.csv")
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [ ]:
#download the data set of anime 
anime_df = pd.read_csv('anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
#chack the data frame size
print(f"anime data set(row, col): {anime_df.shape}\nrating data set(row, col): {rating_df.shape}")

anime data set(row, col): (12294, 7)
rating data set(row, col): (7813737, 3)


In [ ]:
print("Anime:\n")
print(anime_df.info())
print("\n", "*"  * 40)
print("\nRating:\n")
print(rating_df.info())


Anime:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None

 ****************************************

Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


In [ ]:
print("Anime misiing value(%)")
print(round(anime_df.isnull().sum().sort_values(ascending=False)/ len(anime_df.index),4) *100)
print("\n\n", "*" * 50)
print("rating misiing value(%)\n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df.index),4)*100)

Anime misiing value(%)
rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64


 **************************************************
rating misiing value(%)

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [ ]:
print(anime_df['type'].mode())
print(anime_df['genre'].mode())


0    TV
dtype: object
0    Hentai
dtype: object


In [ ]:
#eliminate anime with 0 rating 
anime_df = anime_df[~np.isnan(anime_df["rating"])]

#change NULL to 0 in type and genere
anime_df['type'] = anime_df['type'].fillna(
anime_df['type'].dropna().mode().values[0])

anime_df['genre'] = anime_df['genre'].fillna(
anime_df['genre'].dropna().mode().values[0])

#check null value
anime_df.isnull().sum()


anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [ ]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head(10)

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
5,1,355,NaN
6,1,356,NaN
7,1,442,NaN
8,1,487,NaN
9,1,846,NaN


In [ ]:
#chose only {type = TV}
anime_df = anime_df[anime_df['type'] == 'TV']

#combine the data both anime and rating on the anime ID
rated_anime = rating_df.merge(anime_df, left_on= 'anime_id', right_on= 'anime_id', suffixes= ['_user',''])


#pick *user_id, name, rating from rated_anime
rated_anime = rated_anime[['user_id', 'name', 'rating']]

#For computing reason only we calculate the first 10000 users
rated_anime_10000= rated_anime[rated_anime.user_id <= 10000]
rated_anime_10000.head(10)


,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81
5,21,Naruto,7.81
6,28,Naruto,7.81
7,34,Naruto,7.81
8,38,Naruto,7.81
9,39,Naruto,7.81


In [ ]:
#create a pivot table
pivot = rated_anime_10000.pivot_table(index= 'user_id', columns= 'name', values= 'rating')
pivot.head(10)

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# step 1
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# change Nan value to 0
pivot_n.fillna(0, inplace=True)

# pivot table
pivot_n = pivot_n.T

# Delete the column whchi has the value of  0
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

# using scipy to convert to sparse matrix format for the cosine silimality computation
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [ ]:
#model based on anime similarity
anime_similarity = cosine_similarity(piv_sparse)

#Df of anime similarities
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [ ]:
def anime_recommendation(ani_name):
    
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1  

In [ ]:
anime_recommendation('Naruto')

Recommended because you watched Naruto:

#1: Sword Art Online, 28.65% match
#2: Elfen Lied, 25.81% match
#3: Bleach, 25.39% match
#4: Ao no Exorcist, 24.72% match
#5: Shaman King, 21.2% match
